# Identify working and data directory and site ID.
* The working directory is the root directory of the github.
* The data directory points to the google drive folder where the larger data files are stored.
* For the moment the site ID if the four-digit NEON site ID.

In [3]:
working<-"~/neon_suna"
site_id<-"KING"
data<-paste("~/gdrive/SUNA_Data/SUNA_NEON/",site_id,"/2019",sep="")


# Initialize required libraries, functions and formatting utilities.

In [4]:
library(dplyr)
library(streamMetabolizer)
library(splitstackshape)
library(ggplot2)
library(lubridate)
library(reshape2)
library(neonUtilities)
library(progress)
library(patchwork)
library(doBy)
#library(box)

theme_ts_space<-theme_grey() +
		theme(
#		panel.grid.major = element_blank(),
#		panel.grid.minor = element_blank(),
		panel.background = element_rect(fill="white", colour="black", size=2),
#		legend.key       = element_blank(),
#		legend.text      = element_text(size=20.5),
#		legend.text      = element_blank(),
#		legend.title     = element_text(size=20.5),
		axis.text.x = element_text(size=22,colour="black",hjust=1,angle=45),
		axis.text.y = element_text(size=22,colour="black",vjust=.3),
		axis.title.x = element_text(size=22),
		axis.title.y = element_text(size=22,vjust=-1),
#		plot.title = element_text(hjust = 0.5,size=22,face="bold"),		
#		legend.position  = "left",
		legend.position  = "none",
		plot.margin = unit(c(1,1,1,1), "cm"),
		panel.border = element_rect(colour = "black", fill=NA, size=2)
		)






Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


USGS Active Research Package:
https://owi.usgs.gov/R/packages.html#research

This package was developed for research purposes. We used it for our
own applications and welcome flexible, resilient users who can help us
test and improve the package. Please give us feedback at
https://github.com/USGS-R/streamMetabolizer/issues/new.


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘doBy’


The following object is masked from ‘package:dplyr’:

    order_by




# Setting up the workspace
In the cell below indicate the site that will be analyzed here according to its NEON 4-letter code. A list of files for that site is then placed in the vector "sunas".

If you downloaded the repo into your home director, which is found by going to "~/", then you don"t have to change the box path. FYI: Box is a new package for R that allows you to define a set of functions without creating a new package.

In [4]:

#setwd(paste("~/gdrive/SUNA_data/SUNA_NEON/",site_id,sep=""))
#sunas<-list.files(pattern="*.csv")
#getwd()
#options(box.path = "~/neon_suna/box/")
#box::use(plots)

setwd(data)
sunas<-list.files(pattern="*.CSV")

#suna_cal<-readRDS("/Users/jhosen/gdrive/fdom/SUNA Calibration pdfs/suna_cal_comb.rds")
suna_cal<-read.csv("/Users/jhosen/gdrive/SUNA_Data/SUNA Calibration pdfs/suna_0839_20200312.csv")
suna_cal_t<-as.data.frame(t(suna_cal))
names(suna_cal_t)<-suna_cal_t[1,]

# Data import and processing loop.
This step loads each data file, formats the data for use, and concatenates files from an individual site.

This step could take a while.

In [13]:
head(suna[,c(260:274)])
length(c("suna_id","date","time","nitrate","nitrate_mgl","a254","a350","bromide_trace","spec_average_dark","dark_signal_average","int_time",suna_cal$Wavelength,"sensor_temp","spec_temp","lamp_temp","lamp_time","rel_hum","main_volt","lamp_volt","int_volt","main_current","fit_aux_1","fit_aux_2","fit_base_1","fit_base_2","fit_RMSE","CTD_Time","CTD_Salinity","CTD_Temp","CTD_Pressure","checksum"))

,fit_base_2,fit_RMSE,CTD_Time,CTD_Salinity,CTD_Temp,CTD_Pressure,checksum,NA,NA.1,NA.2,NA.3,NA.4,NA.5,NA.6,NA.7
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
2,6875,6878,6899,6906,6902,6779,6524,6053,9.8,9.8,10.5,162081,0.3,10.7,12.0
3,6900,6864,6895,6893,6894,6784,6519,6044,9.8,9.8,10.5,162082,0.1,10.6,12.0
4,6885,6877,6892,6893,6885,6777,6509,6028,9.8,9.8,10.6,162083,0.0,10.6,12.1
5,6894,6905,6902,6906,6890,6775,6524,6053,9.8,9.8,10.6,162084,0.2,10.7,12.1
6,6890,6885,6881,6903,6913,6808,6540,6074,9.8,9.8,10.6,162085,0.2,10.7,12.1
7,6890,6875,6887,6902,6900,6791,6533,6060,9.8,9.8,10.6,162086,0.0,10.7,12.0


[1] 266

In [17]:
options(warn=-1)
compiled_suna<-data.frame()
sunas_length<-length(sunas)
for(i in 240:sunas_length){
#i<-1
print(i)
suna<-read.csv(paste(sunas[i]),skip=14,header=FALSE)
#head(suna)

#suna_d0<-cSplit(suna,3, ",")
#names(suna)<-c("suna_id","date","time","nitrate","nitrate_mgl","a254","a350","bromide_trace","spec_average_dark","dark_signal_average","int_time",suna_cal$Wavelength,"sensor_temp","spec_temp","lamp_temp","lamp_time","rel_hum","main_volt","lamp_volt","int_volt","main_current","fit_aux_1","fit_aux_2","fit_base_1","fit_base_2","fit_RMSE","CTD_Time","CTD_Salinity","CTD_Temp","CTD_Pressure","checksum")
names(suna)<-c("suna_id","date","time","nitrate","nitrate_mgl","a254","a350","bromide_trace","spec_average_dark","dark_signal_average","int_time",seq(188,395,length.out=256),"sensor_temp","spec_temp","lamp_temp","lamp_time","rel_hum","main_volt","lamp_volt","int_volt","main_current","fit_aux_1","fit_aux_2","fit_base_1","fit_base_2","fit_RMSE","CTD_Time","CTD_Salinity","CTD_Temp","CTD_Pressure","checksum")

#suna<-subset(suna,a254!=0)

suna<-subset(suna,a254!=0)
    
#year<-substr(suna$date, 1, 4)
if(grepl("D2019",sunas[i])==TRUE){
year<-"2019"
doy<-gsub(".CSV","",gsub("D2019","",paste(sunas[i]),fixed=TRUE),fixed=TRUE)
}
    
if(grepl("D2020",sunas[i])==TRUE){
year<-"2020"
doy<-gsub(".CSV","",gsub("D2020","",paste(sunas[i]),fixed=TRUE),fixed=TRUE)
}

day<-as.POSIXct(paste(as.Date(as.numeric(doy)-1,origin=paste(year,"-01-01",sep="")),"00:00:00"),tz="UTC")
#suna$day<-as.POSIXct("2019-05-10 00:00:00",tz="UTC")
suna$dtp<-day + (3600*as.numeric(suna$time))
#attr(suna$dtp,"tzone") <- "Etc/GMT+8"

    

offse<-c(t(suna_cal_t[3,]))
suna2<-(suna[,12:267]/suna$int_time)-suna$dark_signal_average
suna_d0_norm<-sweep(suna2,2,FUN="/",offse)
names(suna_d0_norm)<-paste("n_",names(suna_cal_t),sep="")

    

#print("test")
suna_d0_int<-data.frame()
for(j in 1:nrow(suna_d0_norm)){	
#    print(j)
	flip<-as.data.frame(t(suna_d0_norm[j,]))
	names(flip)<-c("abs")
	flip$wl_nm<-gsub("n_","",row.names(flip),fixed=TRUE)
	flip_int<-as.data.frame(t(approx(flip$wl_nm,flip$abs,xout=seq(189,394,1),rule=2)$y))
	names(flip_int)<-paste("interp_",seq(189,394,1),sep="")
	suna_d0_int<-bind_rows(suna_d0_int,flip_int)
}
suna_d<-bind_cols(suna,suna_d0_int)
#suna_d$date<-as.character(suna_d[,c("date")])
#suna_d$time<-as.character(suna_d[,c("time")])
suna_d2<-suna_d %>% mutate_if(is.numeric,as.character)
compiled_suna<-bind_rows(compiled_suna,suna_d2)
}
print("done")
options(warn=0)    
    
    
    


[1] 240
[1] 241
[1] 242
[1] 243
[1] 244
[1] 245
[1] 246
[1] 247
[1] 248
[1] 249
[1] 250
[1] 251
[1] 252
[1] 253
[1] 254
[1] 255
[1] 256
[1] 257
[1] 258
[1] 259
[1] 260
[1] 261
[1] 262
[1] 263
[1] 264
[1] 265
[1] 266
[1] 267
[1] 268
[1] "done"


# Check Data and Save

In [19]:
#i
grepl("D2020",sunas[240])
sunas[240]
#length(sunas)
#sunas[240]
#ncol(compiled_suna)
nrow(compiled_suna)
#saveRDS(compiled_suna,"/Users/jhosen/gdrive/SUNA_Data/SUNA_NEON/KING/SUNA_KING_20210428_pt2.rds")
compiled_suna<-readRDS("~/neon_suna/data/compiled_suna/SUNA_KING_20210428_full.rds")


[1] TRUE

[1] "D2020001.CSV"

[1] 55079

In [1]:
compiled_suna_1<-readRDS("/Users/jhosen/gdrive/SUNA_Data/SUNA_NEON/KING/SUNA_KING_20210428_pt1.rds")
compiled_suna_2<-readRDS("/Users/jhosen/gdrive/SUNA_Data/SUNA_NEON/KING/SUNA_KING_20210428_pt2.rds")

In [5]:
compiled_suna<-bind_rows(compiled_suna_1,compiled_suna_2)
saveRDS(compiled_suna,"/Users/jhosen/gdrive/SUNA_Data/SUNA_NEON/KING/SUNA_KING_20210428_full.rds")

# Extracting the interpolated columns

In [6]:
compiled_suna$cal<-"suna_0839"

suna_interp<-compiled_suna[,grepl("^inter",names(compiled_suna))] %>% mutate_if(is.character, ~as.numeric(.))
#head(suna_interp)

other_suna<-compiled_suna[,c("suna_id","date","time","nitrate","nitrate_mgl","a254","a350","bromide_trace","spec_average_dark","dark_signal_average","int_time","sensor_temp","spec_temp","lamp_temp","lamp_time","rel_hum","main_volt","lamp_volt","int_volt","main_current","fit_aux_1","fit_aux_2","fit_base_1","fit_base_2","fit_RMSE","CTD_Time","CTD_Salinity","CTD_Temp","CTD_Pressure","checksum","cal","dtp")]

suna_cols<-bind_cols(other_suna,suna_interp)
suna_cols$a254<-as.numeric(suna_cols$a254)
suna_cols$a350<-as.numeric(suna_cols$a350)
suna_cols$nitrate<-as.numeric(suna_cols$nitrate)
suna_cols$nitrate_mgl<-as.numeric(suna_cols$nitrate_mgl)
suna_cols$sensor_temp<-as.numeric(suna_cols$sensor_temp)


In [7]:

a350_mod<-lm(a350~log10(interp_350)+I(log10(interp_350)^2)+I(log10(interp_350)^3),suna_cols)
a254_mod<-lm(a254~log10(interp_254)+I(log10(interp_254)^2)+I(log10(interp_254)^3),suna_cols)



cor_cols<-grep("^interp_",names(suna_cols))


Warning message in eval(predvars, data, env):
“NaNs produced”
Warning message in structure(x, class = unique(c("AsIs", oldClass(x)))):
“NaNs produced”
Warning message in structure(x, class = unique(c("AsIs", oldClass(x)))):
“NaNs produced”
Warning message in eval(predvars, data, env):
“NaNs produced”
Warning message in structure(x, class = unique(c("AsIs", oldClass(x)))):
“NaNs produced”
Warning message in structure(x, class = unique(c("AsIs", oldClass(x)))):
“NaNs produced”


In [8]:

#compiled_suna2[,c(cor_cols)]<-compiled_suna2[,c(cor_cols)] %>% mutate_if(is.character,as.numeric)

for(i in 1:length(cor_cols)){
	prepdata<-data.frame(interp_350=suna_cols[,cor_cols[i]],interp_254=suna_cols[,cor_cols[i]])
	a350_pred<-predict(a350_mod,newdata=prepdata)
	a254_pred<-predict(a254_mod,newdata=prepdata)
	suna_cols[,c(paste(names(suna_cols)[cor_cols[i]],"_c350",sep=""))]<-a350_pred
	suna_cols[,c(paste(names(suna_cols)[cor_cols[i]],"_c254",sep=""))]<-a254_pred
}



Warning message in eval(predvars, data, env):
“NaNs produced”
Warning message in structure(x, class = unique(c("AsIs", oldClass(x)))):
“NaNs produced”
Warning message in structure(x, class = unique(c("AsIs", oldClass(x)))):
“NaNs produced”
Warning message in eval(predvars, data, env):
“NaNs produced”
Warning message in structure(x, class = unique(c("AsIs", oldClass(x)))):
“NaNs produced”
Warning message in structure(x, class = unique(c("AsIs", oldClass(x)))):
“NaNs produced”
Warning message in eval(predvars, data, env):
“NaNs produced”
Warning message in structure(x, class = unique(c("AsIs", oldClass(x)))):
“NaNs produced”
Warning message in structure(x, class = unique(c("AsIs", oldClass(x)))):
“NaNs produced”
Warning message in eval(predvars, data, env):
“NaNs produced”
Warning message in structure(x, class = unique(c("AsIs", oldClass(x)))):
“NaNs produced”
Warning message in structure(x, class = unique(c("AsIs", oldClass(x)))):
“NaNs produced”
Warning message in eval(predvars, data, 

# Reducing data down to 15 minutes intervals.
We want to average down each burst from the SUNA to a single value and mac sure that these values are snapped to 15 minutes intervals (e.g., rather than round up or down to 14 or 16).

In [27]:
str(suna_cols$sensor_temp)

 num [1:570932] 9.3 9.3 9.3 9.3 9.3 9.3 9.3 9.4 9.4 9.4 ...


In [ ]:
rm(compiled_suna)
rm(compiled_suna_1)
rm(compiled_suna_2)

suna_cols$dtpr<-lubridate::round_date(suna_cols$dtp, "15 minutes")
suna_red<-summaryBy(.~dtpr,suna_cols,FUN=c(mean))



# Downloading turbidity time series data from NEON.
Data product DP1.20288.001 for general water quality sonde data.

In [30]:
CARI_wqs<-loadByProduct(dpID="DP1.20288.001",site="KING",check.size=F)
saveRDS(CARI_wqs,paste(data,"/KING_wqs.rds",sep=""))
CARI_wqs<-readRDS(paste(data,"/KING_wqs.rds",sep=""))


Finding available files
  |======================================================================| 100%

  |======================================================================| 100%

Unpacking zip files using 1 cores.
Stacking operation across a single core.
Stacking table waq_instantaneous
Stacking table ais_maintenance
Copied the most recent publication of variable definition file to /stackedFiles
Finished: Stacked 2 data tables and 1 metadata tables!
Stacking took 4.662895 mins


In [ ]:
CARI_wqsd<-CARI_wqs$waq_instantaneous
CARI_wqsd$dtp<-CARI_wqsd$startDateTime
CARI_wqk<-subset(CARI_wqsd[,c("siteID","dtp","specificConductance","dissolvedOxygen","pH","chlorophyll","turbidity","fDOM")],!is.na(CARI_wqsd$specificConductance))
#str(CARI_wqk)

CARI_wqk$dtpr<-lubridate::round_date(CARI_wqk$dtp, "15 minutes")
CARI_wq_red<-summaryBy(.~dtpr,CARI_wqk,FUN=c(mean))
str(CARI_wq_red)

In [ ]:
CARI_sw<-merge(suna_red,CARI_wq_red,by="dtpr",all.x=TRUE)


# import and merge NEON water quality grab sample data.

In [ ]:


library(zoo)
#CARI_wqg<-loadByProduct(dpID="DP1.20093.001",site="CARI",check.size=F)
#saveRDS(CARI_wqg,paste(data,"/CARI_wqg.rds",sep=""))
CARI_wqg<-readRDS(paste(data,"/CARI_wqg.rds",sep=""))

CARI_wqg_d<-as.data.frame(CARI_wqg$swc_externalLabDataByAnalyte)
uv_abs<-subset(CARI_wqg_d,analyte=="UV Absorbance (250 nm)"|analyte=="UV Absorbance (280 nm)")
date_cast<-dcast(uv_abs[,c("analyte","collectDate","analyteConcentration")],collectDate~analyte,value.var="analyteConcentration",mean)
date_cast$dtpr<-lubridate::round_date(date_cast$collectDate, "15 minutes")

suna_grab<-merge(CARI_sw,date_cast,by="dtpr")

#temp_nitrate_zoo<-zoo(nitrate$nitrate_umL,nitrate$dtp)
#temp_n<-na.approx(temp_nitrate_zoo,xout=suna_red$dtp,na.rm=FALSE)
#suna_grab[,c("UV Absorbance (250 nm)")]
names(suna_grab)<-gsub("UV Absorbance (250 nm)","uva_250_lab",names(suna_grab),fixed=TRUE)
names(suna_grab)<-gsub("UV Absorbance (280 nm)","uva_280_lab",names(suna_grab),fixed=TRUE)

suna_grab[,c("dtpr","a254.mean","uva_250_lab")]


# Exploring turbidity corrections

In [ ]:
names(suna_grab)

In [ ]:
suna_grab$interp_250_log<-log10(suna_grab$interp_250_c254.mean)
suna_grab$turb_log<-log10(suna_grab$turbidity.mean)


#summary(lm(uva_250_lab~interp_250_log*turb_log,suna_grab))
#summary(lm(uva_250_lab~interp_250_c254.mean*turb_log,suna_grab))

suna_grab$interp_250_tcorr<-predict(lm(uva_250_lab~interp_250_c254.mean*turbidity.mean,suna_grab))
suna_grab$interp_250_tcorr_log<-predict(lm(uva_250_lab~interp_250_log*turbidity.mean,suna_grab))
suna_grab$interp_250_tcorr_log_turb<-predict(lm(uva_250_lab~interp_250_log*turb_log,suna_grab))


ggplot(suna_grab,aes(uva_250_lab,interp_250_c254.mean,color=pH.mean))+
theme_ts_space+
xlab("UV Absorbance at 250 nm (Laboratory)")+
ylab("UV Absorbance at 250 nm (SUNA Uncorrected)")+
geom_point(size=6)+
ggtitle("Uncorrected")

ggsave("~/neon_suna/plots/CARI_250_uncorrected.pdf",width = 20, height = 20, units = "cm")



ggplot(suna_grab,aes(uva_250_lab,interp_250_tcorr_log,color=pH.mean))+
theme_ts_space+
geom_smooth(method="lm",color="grey20")+
xlab("UV Absorbance at 250 nm (Laboratory)")+
ylab("UV Absorbance at 250 nm (SUNA Corrected)")+
geom_point(size=6)+
ggtitle("Turbidity Corrected")

ggsave("~/neon_suna/plots/CARI_250_corrected.pdf",width = 20, height = 20, units = "cm")




#summary(lm(uva_250_lab~interp_250_c254.mean*turbidity.mean+sensor_temp.mean,suna_grab))
#summary(lm(uva_280_lab~interp_280_c350.mean*turbidity.mean+sensor_temp.mean,suna_grab))
#summary(lm(uva_280_lab~interp_280_c254.mean*turbidity.mean+sensor_temp.mean,suna_grab))

In [ ]:
names(CARI_sw)

# Apply turbidity correction based on lab samples.

In [ ]:
cor_cols<-grep("^interp_",names(CARI_sw))

uva_250_turb_lm<-lm(uva_250_lab~interp_250_log*turbidity.mean,suna_grab)

for(i in 1:length(cor_cols)){
	prepdata<-data.frame(interp_250_log=log10(CARI_sw[,cor_cols[i]]),turbidity.mean=CARI_sw$turbidity.mean)
	turb_cor_pred<-predict(uva_250_turb_lm,newdata=prepdata)
	CARI_sw[,c(paste(names(CARI_sw)[cor_cols[i]],"_turb",sep=""))]<-turb_cor_pred
}



In [ ]:
head(CARI_sw)

In [ ]:
CARI_swt_0<-CARI_sw[,grep("*_turb$",names(CARI_sw))]
CARI_swt<-bind_cols(CARI_sw[,c("dtpr","nitrate.mean","nitrate_mgl.mean","a254.mean","a350.mean","sensor_temp.mean","specificConductance.mean","dissolvedOxygen.mean","pH.mean","chlorophyll.mean","turbidity.mean","fDOM.mean")],CARI_swt_0)





# Adding PAR data.

In [ ]:
CARI_par<-loadByProduct(dpID="DP1.20042.001",site="CARI",check.size=F)
saveRDS(CARI_par,paste(getwd(),"/CARI_par.rds",sep=""))
CARI_par<-readRDS(paste(getwd(),"/CARI_par.rds",sep=""))




In [ ]:
str(CARI_par)

In [ ]:

CARI_parts<-CARI_par$PARWS_5min
CARI_parts$dtp<-CARI_parts$startDateTime
#str(CARI_wqk)



CARI_parts$dtpr<-lubridate::round_date(CARI_parts$dtp, "15 minutes")
CARI_parts_red<-summaryBy(PARMean~dtpr,CARI_parts,FUN=c(mean))
head(CARI_parts_red)
#str(CARI_wq_red)

CARI_swtpar<-merge(CARI_swt,CARI_parts_red,by="dtpr",all.x=TRUE)


In [ ]:

str(CARI_swtpar$dtpr)

attr(CARI_swtpar$dtpr,"tzone") <- "Etc/GMT+5"

str(CARI_swtpar$dtpr)
CARI_swtpar$date<-as.Date(CARI_swtpar$dtpr,tz="Etc/GMT+5")

CARI_dates<-unique(CARI_swtpar$date)

for(i in 1:length(CARI_dates)){
    
    
}

In [ ]:

#CARId<-subset(CARI_swtpar,date==as.Date("2019-06-10"))
CARId<-CARI_swtpar

CARIdk<-CARId[,grep("*c254.mean_turb$",names(CARId))]
CARIdk$dtpr<-CARId$dtpr

CARIdm<-subset(melt(CARIdk,id.vars=c("dtpr")),!is.na(value))
CARIdm$wavelength<-gsub("interp_","",CARIdm$variable,fixed=TRUE)
CARIdm$wavelength<-as.numeric(gsub("_c254.mean_turb","",CARIdm$wavelength,fixed=TRUE))



str(CARIdm)

# Trying some ridge plots to look at wavelength changes over time.
So far not looking super great.

In [ ]:
library(ggridges)
#scales::rescale(height)
CARIdm$dtprn<-scales::rescale(as.numeric(CARIdm$dtpr))
CARIdm$abs<-scales::rescale(as.numeric(CARIdm$value))


ggplot(subset(CARIdm,wavelength>=275&wavelength<=295),aes(x = wavelength, y = dtprn, group=dtprn,height = abs)) +
  geom_ridgeline(fill="grey80",alpha=0.6)

ggsave("~/neon_suna/plots/CARIdm_ridge.pdf",width = 20, height = 40, units = "cm")


# Calculating spectral slope 275-295nm on each time step.

In [ ]:
names(CARId)
names(CARIdm)

In [ ]:
ssm_275_295<-subset(CARIdm,wavelength>=275&wavelength<=295)
dtps<-unique(ssm_275_295$dtpr)
pb <- progress_bar$new(
	format = "  downloading [:bar] :percent eta: :eta",
	total = length(dtps), clear = FALSE, width= 60)
s275295_comp<-data.frame()


for(i in 1:length(dtps)){
	ssm_now<-subset(ssm_275_295,dtpr==dtps[i])
	
	ssm_now$am1<-ssm_now$value*100*2.3025851
	ssm_now$lnam1<-log(ssm_now$am1)


	if(sum(!is.na(ssm_now$lnam1))>2){
		s275295<-lm(ssm_now$lnam1~ssm_now$wavelength)$coefficients[2]*-1
		}else{s275295<-NA}
		s275295_temp<-data.frame(dtpr=dtps[i],s275295=s275295)
		s275295_comp<-bind_rows(s275295_comp,s275295_temp)
#		pb$tick()
}	
	


In [ ]:
nrow(s275295_comp)
nrow(CARI_swtpar)
CARI_swtp_ss<-merge(CARI_swtpar,s275295_comp,by="dtpr",all.x=TRUE)
saveRDS(CARI_swtp_ss,paste(data,"CARI_swtp_ss.rds",sep=""))

# Extracting spectral daily slope ratio discrepancy.

In [ ]:
sdates<-unique(as.Date(CARI_swtp_ss$dtpr,tz="Etc/GMT+5"))

#i<-30
comp_df<-data.frame()
for(i in 1:length(sdates)){
#print(i)
start<-as.POSIXct(paste(sdates[i]-1,"22:00",tz="Etc/GMT+5"))
finish<-as.POSIXct(paste(sdates[i]+1,"03:00",tz="Etc/GMT+5"))
CARIday<-subset(CARI_swtp_ss,dtpr>=start & dtpr<=finish)
CARIday<-subset(CARIday,!is.na(PARMean.mean) & !is.na(s275295))

if(nrow(CARIday)>80){
    CARIday$PAR_roll<-as.numeric(c("NA",rollmean(CARIday$PARMean.mean,k=3,align=c("center"),na.fill=TRUE),"NA"))
    CARIday$s275295_roll<-as.numeric(c("NA",rollmean(CARIday$s275295,k=3,align=c("center"),na.fill=TRUE),"NA"))



    CARI_night<-subset(CARIday,PARMean.mean<200)
    night_s275295<-mean(CARI_night$s275295,na.rm=TRUE)
    
    #CARIday$baseline<-predict(lm(s275295~dtpr,CARI_night),CARIday)
    CARIday$baseline_roll<-predict(lm(s275295_roll~dtpr,CARI_night),CARIday)    
    CARIday$s275295_bs<-CARIday$s275295_roll-CARIday$baseline_roll    
    
    CARI_night_am<-subset(CARI_night,hour(dtpr)<=12)
    night_s275295_am<-mean(CARI_night_am$s275295,na.rm=TRUE)
    
    CARI_night_pm<-subset(CARI_night,hour(dtpr)>12)
    night_s275295_pm<-mean(CARI_night_pm$s275295,na.rm=TRUE)
    
    CARI_day<-subset(CARIday,PARMean.mean>=10)
    day_s275295<-mean(CARI_day$s275295,na.rm=TRUE)

    day_s275295_base_sum<-sum(CARI_day$s275295_bs,na.rm=TRUE)
    day_PAR_roll_base_sum<-sum(CARI_day$PAR_roll,na.rm=TRUE)    

    int<-lm(s275295_bs~PAR_roll,CARI_day)$coef[1]
    slope<-lm(s275295_bs~PAR_roll,CARI_day)$coef[2]    
    
    CARI_n_am_time<-nrow(CARI_night_am)
    CARI_n_pm_time<-nrow(CARI_night_pm)
    CARI_d_time<-nrow(CARI_day)
    
    temp_df<-data.frame(site="CARI",date=sdates[i],night_s275295=night_s275295,night_s275295_am=night_s275295_am,night_s275295_pm=night_s275295_pm,day_s275295=day_s275295,day_s275295_base_sum=day_s275295_base_sum,day_PAR_roll_base_sum=day_PAR_roll_base_sum,int=int,slope=slope,CARI_n_am_time=CARI_n_am_time,CARI_n_pm_time=CARI_n_pm_time,CARI_d_time=CARI_d_time)
    comp_df<-bind_rows(comp_df,temp_df)
    }
}




In [ ]:
    CARI_day$s275295_bs
ggplot(comp_df,aes(date,day_s275295_base_sum))+
       geom_point(size=2)
ggplot(comp_df,aes(date,day_PAR_roll_base_sum))+
       geom_point(size=2)


ggplot(comp_df,aes(day_PAR_roll_base_sum,day_s275295_base_sum))+
geom_point()


saveRDS(comp_df,paste(data,"comp_df_cari.rds",sep=""))

In [ ]:
CARIday<-subset(CARI_swtp_ss,as.Date(dtpr,tz="Etc/GMT+5")==as.Date("2019-06-11",tz="Etc/GMT+5"))
nrow(CARIday)

str(CARIday$s275295)



ggplot(CARIday,aes(dtpr,s275295))+
geom_point()


ggplot(CARIday,aes(dtpr,PARMean.mean))+
geom_point()

